In [ ]:
import torch
from torch.utils.data import Dataset

工具函数 , 用来 decode 部分的掩码创建 . 

triu 会创建将一个 tensor 的对角线上方元素保留 , 对角线本身以及其下方的元素清零 . 

而返回时则又用 == 0 返回一个布尔遮罩 , 值为零的元素所在位置被赋值为真 . 

In [ ]:
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

数据集的创建 , 虽然看着复杂 , 但总归是 `__init__` , `__len__` , `__getitem__` 三个函数之间的事

在 train.py 部分 , ds_raw 是一个从 hugging face 来的 datasets.Dataset ( 注意不是 torch 提供的 Dataset ) , 其经过 torch 的 random_split 后变为 torch.utils.data.dataset.Subset 这一数据类型 ( 类似于 Dataset , 但本质是一个索引列表 , 也能提供 `__len__` 与 `__getitem__` ) .

但注意这里虽说 raw 相关数据是能当作 Dataset , 但它没有分词 , 没有特殊符号的嵌入 , 没有 label 输出 , 只能返回原始数据的字典 , 基本没法用 , 所以还需要 bilingualdataset 进一步封装 . ( 原始数据差不多像这种东西 : `{'translation': {'en': "Hello world!", 'it': "Ciao mondo!"}}` ) 

初始化同时单独拿出来了三个特殊 token 方便后面插入使用 ( 毕竟当前它们只是在字典里存在 , 但在语料里没有真的被插入过 ) . 而 `token_to_id` 外面又单独用 `[]` 改成一个列表 , 这是因为 `.tensor()` 接受的应该是一个列表类的东西 . 

`__len__` 方法可以直接借用其原始 subset 提供的 `__len__` , 也就是 Dataset 的元素数目 ( 但是数量却用 len 描述 , 就很奇怪 )

`__getitem__` 部分 : 首先其本身目的效果是使能接收 idx , 并对应地返回单套的数据 , 

因而首先可以利用 subset 的 `__getitem__` 方法获得单套原始数据 , 在其基础上进行包装 , 具体获得就只需要像数组访问一样用中括号 , 而得到的是上面展示过的数据 ( 这对后面理解获取某项数据 , 添加特殊字符等有帮助 )

获得原始 text 后就可以进一步用之前配置好的 tokenizer 进行 tokenize . 而其会得到一个含各种数据的 Encoding 对象 , 需要用 `.ids` 得到模型需要的数字 id 序列 , 数据类型为列表 .

而接下来需要对句子进行进一步处理 : 补充特殊字符 . 首先计算了两种句子的需要补齐 padding 数量 : 其中因为 enc 需要加入 `[SOS]` 与 `[EOS]` 而需要减二 , dec 只需要加入 `[SOS]` 而减一 .

如果句子太长的话需要截断 . 但由于这里使用的 seq_len 是原数据集的最大句子长度 , 所以一旦还有数量问题一定是出错了 , 就跳过了截断过程 . 
 
最后拼接 , 每个句子插入需要的 `[SOS]` , `[EOS]` 字符 , 并在后面按需加入 `[PAD]` . 这种拼接在第零维发生 , 也就是说输出的还是一个 tensor " 向量 " , 大小为 ( seq_len , )

enc 与 dec 的插入逻辑已经写过 . 而 label 则是一个以 `[EOS]` 结尾的序列 , 作为序列后面用时会将其每个字符作为一个预测目标 ( 预测 `label[0]` , `label[1]` ... ) 从而一套数据每次被用来训练时会进行 label 有效 token 数量的训练 , 它们的总和记为这个单套的 loss , 与简单的分类问题还是有层级上的去别的 .

( 具体逐个学习的过程 , 可以看 train.py 文件中的相关笔记 )

最后的 mask 与 unsqueeze 又会涉及到维度操作 . 

encoder_mask 部分 , 将原本形如 `[SOS, token1, token2, ..., EOS, PAD, PAD, ...]` 的句子对应生成形如 `tensor([[[1, 1, 1, ..., 1, 0, 0]]], dtype=torch.int32)` , 的尺寸为 ( 1, 1, seq_len ) 的张量 , 为的是匹配维度后方便广播 ( 一个 batch 内有同尺寸的一系列 mask , 但是它们的底层样子因为 padding 情况不同而不一样 ) 

decoder_mask 部分类似 , 但其通过与三角遮罩再过一遍时也能顺便完成广播 . 具体来讲是 ( 1, seq_len ) & ( 1, seq_len, seq_len ) --> ( 1, seq_len, seq_len ) , 具体二者的维度匹配可以看训练函数 . 

而这些信息整体一起打包返回 , 也就方便后面使用了 . 

In [ ]:
class BilingualDataset(Dataset):

    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len):
        super().__init__()
        self.seq_len = seq_len

        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

        self.sos_token = torch.tensor([tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_tgt.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_tgt.token_to_id("[PAD]")], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        src_target_pair = self.ds[idx]
        src_text = src_target_pair['translation'][self.src_lang]
        tgt_text = src_target_pair['translation'][self.tgt_lang]

        enc_input_tokens = self.tokenizer_src.encode(src_text).ids
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2 
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1

        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError("Sentence is too long")

        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(enc_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        decoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            "encoder_input": encoder_input,  
            "decoder_input": decoder_input,  
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(), # (1, 1, seq_len)
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).int() & causal_mask(decoder_input.size(0)), # (1, seq_len) & (1, seq_len, seq_len),
            "label": label,  
            "src_text": src_text,
            "tgt_text": tgt_text,
        }